# Hardware Compatible Gradient Computation on Quantum Hardware

This notebook explores the performance of different gradient computation techniques for quantum hardware.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import qnetvo as qnet

In [2]:
from qiskit import IBMQ

# For details regarding integration between PennyLane and IMB Q,
# see https://pennylaneqiskit.readthedocs.io/en/latest/devices/ibmq.html#accounts-and-tokens
provider = IBMQ.load_account()

## Setup

We prepare a static bell state and optimize the CHSH violation over measurements in the $xz$-plane. 

In [3]:
prep_nodes = [
    qnet.PrepareNode(1, [0,1], qnet.ghz_state, 0)
]
meas_nodes = [
    qnet.MeasureNode(2, 2, [0], qnet.local_RY, 1),
    qnet.MeasureNode(2, 2, [1], qnet.local_RY, 1)
]

We use the `defualt.qubit` device to evaluate the cost function.

In [4]:
local_chsh_ansatz = qnet.NetworkAnsatz(prep_nodes, meas_nodes)
chsh_cost = qnet.chsh_inequality_cost(local_chsh_ansatz)

We test the gradients computations on the IBM hwardware simulator and the `ibmq_belem` device.

In [5]:
dev_ibm_qasm = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_qasm_simulator",
    "provider" : provider
}
dev_ibm_belem = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_belem",
    "provider" : provider
}

ibm_sim_chsh_ansatz = qnet.NetworkAnsatz(
    prep_nodes, meas_nodes, dev_kwargs = dev_ibm_qasm
)
belem_chsh_ansatz = qnet.NetworkAnsatz(
    prep_nodes, meas_nodes, dev_kwargs = dev_ibm_belem
)

## Simulator Runs

We first demonstrate three different gradients on the IBM hardware simulator including parameter shift rule, finite differences rule, and the quantum natural gradient.
In each of these cases, the gradients are computed using 4 parallelized threads.

### Parameter Shift Rule

In [6]:
%%time

param_shift_grad = qnet.parallel_chsh_grad(
    ibm_sim_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

param_shift_opt = qnet.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=param_shift_grad
)

print("max score : ", param_shift_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
elapsed time :  173.7887988090515
max score :  1.9598712676028576
CPU times: user 1.52 s, sys: 162 ms, total: 1.68 s
Wall time: 2min 54s


### Quantum Natural Gradient

This approach requires 4 extra calls to the quantum hardware resulting in an increased overhead, however, it may show some optimization advantages over other approaches in practice on noisy hardware.

In [7]:
%%time

natural_grad = qnet.chsh_natural_grad(
    ibm_sim_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

natural_grad_opt = qnet.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=natural_grad
)

print("max score : ", natural_grad_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
elapsed time :  203.55724000930786
max score :  2.1202011744620237
CPU times: user 1.48 s, sys: 103 ms, total: 1.59 s
Wall time: 3min 23s


## Quantum Hardware Runs

We now demonstrate the same optimization step on IBM quantum computers.

### Parameter Shift Rule

In [8]:
%%time

param_shift_grad = qnet.parallel_chsh_grad(
    belem_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

belem_param_shift_opt = qnet.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=param_shift_grad
)

print("max score : ", belem_param_shift_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905


IBMQJobInvalidStateError: 'Unable to retrieve result for job 622aa174817d6115e0a884e0. Job was cancelled.'

### Quantum Natural Gradient

In [10]:
%%time

natural_grad = qnet.chsh_natural_grad(
    belem_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

belem_natural_grad_opt = qnet.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=natural_grad
)

print("max score : ", belem_natural_grad_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905


KeyboardInterrupt: 